# Plot bifurcations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle5 as pickle
import glob
import csv
import os

In [ ]:
#### Change to match your directories and select if you want the AV segmentation:
dir_images='/Users/sortinve/Desktop/Vascular_shared_genetics_in_the_retina/__CODIGO/retina-phenotypes/input/DRIVE_images/DRIVE/'
dir_ARIA_output= '/Users/sortinve/Desktop/Vascular_shared_genetics_in_the_retina/__CODIGO/retina-phenotypes/output/ARIA_output_DRIVE/'
dir_bif = '/Users/sortinve/Desktop/Vascular_shared_genetics_in_the_retina/__CODIGO/retina-phenotypes/output/phenotypes_DRIVE_all/bifurcations_position/'
with_AV_segmentation=True #True or False

In [ ]:
os.chdir(dir_images) 

for img in glob.glob("*.png"):
    try:
        imageID = os.path.splitext(img)[0]
        num=imageID.split("_")[0]
        
        fig2 = plt.figure(figsize = (5,5)) # create a 5 x 5 figure 
        ax3 = fig2.add_subplot(111)
        ax3.imshow(mpimg.imread(img), interpolation='none')

        if with_AV_segmentation:
            X = []
            Y = []
            segmentStats = []

            with open(dir_ARIA_output+ imageID + "_all_center2Coordinates.tsv") as fd:
                rd = csv.reader(fd, delimiter='\t')
                X.extend([float(j) for j in row] for row in rd)
            with open(dir_ARIA_output+ imageID + "_all_center1Coordinates.tsv") as fd:
                rd = csv.reader(fd, delimiter='\t')
                Y.extend([float(j) for j in row] for row in rd)
            with open(dir_ARIA_output+ imageID + "_all_segmentStats.tsv") as fd:
                rd = pd.read_csv(fd, sep='\t')
                segmentStats = rd["AVScore"]

            df = pd.DataFrame([])
            df["segmentStats"] = segmentStats

            df_results = pd.DataFrame([])
            df_aux = pd.DataFrame([])
            aux = int(df.count(axis=0))

            # 'Arteries' if df['AVScore'] > 0
            # 'Veins' if df['AVScore'] < 0
            for i in range(aux):
                df_aux = pd.DataFrame(X[i])
                df_aux["Y"] = pd.DataFrame(Y[i])
                df_aux["type"] = segmentStats[i]
                df_aux["i"] = i
                df_results = df_results.append(df_aux, True)

            df_results.columns = ['X', 'Y', 'type', 'i']
            df_results['type'] = np.sign(df_results['type'])

            # Delete segments that were not identifiy as an artery or vein:
            df_results = df_results.loc[df_results['type'] != 0]

            plt.scatter(x=df_results['X'], y=df_results['Y'], c=df_results['type'], cmap="jet", marker="d", alpha=0.5, s= 0.2)

        bif_positions = pd.read_csv(dir_bif + num + '_training_bifurcations_position.csv')
        plt.scatter(bif_positions['Y'], bif_positions['X'], color='black', s=6)

        plt.title(f'{str(img)} N bifurcations: {len(bif_positions)}')
        plt.show()
    except Exception as e:
        print(e)